# CSTU Project for "Computer Vision"
Jan 25 2020
# CNN for gesture prediction

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.0.0


If tensorflow version is not 2.0 or later, uninstall tensorflow, and re-install with version 2.0 or later.

In [15]:
!pip install tensorflow==2.0.0

In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
"""
In the raw csv file, labels are coded in integer.  This dictionary translates integer
codes into alphabets
"""
translate={'01':'paper','02':'scissors','03':'rock'}

In [3]:
import csv
data_dir='images/'
with open(data_dir+'label_file.csv') as f:
    reader = csv.reader(f)
    files_list = list(reader)

FileNotFoundError: [Errno 2] No such file or directory: 'images/label_file.csv'

In [4]:
files_list=files_list[1:]

NameError: name 'files_list' is not defined

In [22]:
data_dir='images/'
training_dir=data_dir+'training/'
validate_dir=data_dir+'validate/'

The input images will be in gray scale with 1 channel (instead of 3).  The shape of images will be (width,height,1)

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,)
test_datagen = ImageDataGenerator(rescale=1./255,)
height=50
width=50
drop_rate=0.3

# Flow training images in batches of 20 using train_datagen generator
train_generator = test_datagen.flow_from_directory(
        training_dir,  # This is the source directory for training images
        target_size=(height,width),  # All images will be resized to 50x50
        color_mode='grayscale',
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = train_datagen.flow_from_directory(
        validate_dir,
        target_size=(height,width),
        color_mode='grayscale',
        batch_size=20,
        class_mode='categorical')

Found 1823 images belonging to 3 classes.
Found 456 images belonging to 3 classes.


In [33]:
train_generator.class_indices

{'paper': 0, 'rock': 1, 'scissors': 2}

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cProfile

In [46]:
def my_model():
  classes=len(train_generator.class_indices)
  model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(height,width, 1)),
    MaxPooling2D(2, 2),
    Dropout(drop_rate),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(drop_rate),  
    Dense(128, activation='relu'),
    Dense(classes, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [47]:
model = my_model()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 7744)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 7744)             

In [48]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,  # 2000 images = batch_size * steps = 20*100
      epochs=5,
      validation_data=validation_generator,
      validation_steps=20,  # 1600 images = batch_size * steps = 20*80
      verbose=2)

Epoch 1/5
50/50 - 33s - loss: 1.0969 - accuracy: 0.3937 - val_loss: 1.0492 - val_accuracy: 0.4425
Epoch 2/5
50/50 - 34s - loss: 0.8797 - accuracy: 0.6250 - val_loss: 0.5897 - val_accuracy: 0.8675
Epoch 3/5
50/50 - 33s - loss: 0.3996 - accuracy: 0.8627 - val_loss: 0.3410 - val_accuracy: 0.8650
Epoch 4/5
50/50 - 33s - loss: 0.2215 - accuracy: 0.9250 - val_loss: 0.1593 - val_accuracy: 0.9575
Epoch 5/5
50/50 - 33s - loss: 0.1517 - accuracy: 0.9540 - val_loss: 0.1187 - val_accuracy: 0.9625


In [49]:
# model save
model.save('model_test.h5')